# Domain-specific languages in Julia: Generating callable objects using macros

Since the beginning of Julia, it has been tempting to use macros to write **domain-specific languages** (DSLs), i.e. to *extend* Julia syntax to provide a simpler interface to create Julia objects with complicated behaviour. The first, and still most extensive, example is [JuMP](https://github.com/JuliaOpt/JuMP.jl).

Since the fix for the infamous early Julia issue [number #265](https://github.com/JuliaLang/julia/issues/265), which was incorporated in Julia 0.6, released a week ago, some previous methods for creating DSLs in Julia, mainly involving `eval`, ceased to work.

In this post, we will describe a pattern (i.e., a reusable structure) for creating DSLs *without* the use of `eval`; this is the recommended solution.

We will use syntax suitable for Julia 0.6 and later versions. We strongly recommend that everybody upgrade to Julia 0.6.

## Creating a `Model` object containing a function

This blog post arose from a question in the JuliaCon 2017 hackathon about the [Modia modelling language](https://github.com/ModiaSim/Modia.jl), where there is a `@model` macro. Here we will describe the simplest possible version of such a macro, which will create a `Model` object that contains a function, and is itself callable.

First we define the `Model` object. It is tempting to write it like this:

In [ ]:
struct NaiveModel
    f::Function
end

We can then create an instance of the `NaiveModel` type (i.e. an object of that type) using the default constructor, e.g. by passing it an anonymous function:

In [ ]:
m1 = NaiveModel(x->2x)

and we can call the function using

In [ ]:
m1.f(10)

If we wish instances like `m` to themselves behave like functions, we can overload the call syntax on the `NaiveModel` object:

In [ ]:
(m::NaiveModel)(x) = m.f(x)

so that we can now just write

In [ ]:
m1(10)

## Parametrising the type

Since `Function` is an abstract type, for performance we should *not* have a field of this type inside our object.
Rather, we parametrise the type using the type of the function:

In [ ]:
struct Model{F}
    f::F
end

(m::Model)(x) = m.f(x)

In [ ]:
m2 = Model(x->2x)

In [ ]:
m2(10)

Let's compare the performance:

In [ ]:
using BenchmarkTools

In [ ]:
@btime m1(10)

In [ ]:
@btime m2(10)

Indeed we have removed some overhead in the second case.

## Defining a macro to create objects

We now wish to define a macro that will allow us to use a simple syntax (that uses standard Julia syntax but itself is not valid Julia code) to create objects. In this case, we wish to write

    @model 2x
    
to define a `Model` object with the function `x->2x`.

We will build up to macros by first building the tools (standard Julia functions) to manipulate the expression `2x` in the correct way to build a `Model` object from it.

First, let's create a function to manipulate our expression:

In [ ]:
function make_function(ex::Expr)
    return :(x -> $ex)
end

In [ ]:
ex = :(2x)
make_function(ex)

This function assumes that `ex` is an expression containing the variable `x` and makes a new expression representing an
anonymous function with the single argument `x`. (See [my JuliaCon 2017 tutorial](https://github.com/dpsanders/julia_towards_1.0/blob/master/4.%20Metaprogramming.ipynb) for an example of how to walk through the expression tree in order to extract *automatically* the variables that it contains.)

Now let's define a function `make_model` that takes a function and wraps it and passes it into a `Model` object:

In [ ]:
function make_model(ex::Expr)
    return :(Model($ex))
end

In [ ]:
make_model(make_function(:(2x)))

If we evaluate this "by hand", we see that it correctly creates a `Model` object:

In [ ]:
m3 = eval(make_model(make_function(:(2x))))

In [ ]:
m3(10)

## Making a macro

However, this is ugly and clumsy. Instead, we now wrap everything inside a **macro**.

A **macro** is a kind of "super-function" that manipulates code. 


A macro, in the simplest case, takes a single Julia `Expr` object (i.e. an unevaluated Julia expression) as argument. 
It manipulates this expression object to create a new expression object, and returns this new expression.

The key point is that **this returned expression is *automatically* evaluated in the correct scope** when the macro returns.
Indeed, the macro actually "splices in" the newly-generated code in place of the old code, before the Julia compiler kicks in.

Let's start with the simplest possible macro:

In [ ]:
macro model(ex)
    @show ex
    @show typeof(ex)
    return nothing
end

It just shows the argument that it was passed and exits, returning an empty expression.

In [ ]:
m4 = @model 2x

We see that the Julia `Expr` object has been automatically created from the explicit code that we typed.

Now we can plug in our previous functions to complete the macro's functionality:

In [ ]:
macro model(ex)
    return make_model(make_function(ex))
end

In [ ]:
m5 = @model 2x

In [ ]:
@macroexpand @model 2x

In [ ]:
m5(10)

To check that the macro is doing what we think it is, we can use the `@macroexpand` command, which itself is a macro (as denoted by the initial `@`):

In [ ]:
@macroexpand @model 2x

## Macro "hygiene"

However, our macro has an issue, called macro "hygiene". This has to do with where variables are defined. Let's put everything we have so far inside a module:

In [ ]:
module Models

export Model, @model

struct Model{F}
    f::F
end

(m::Model)(x) = m.f(x)

function make_function(ex::Expr)
    return :(x -> $ex)
end

function make_model(ex::Expr)
    return :(Model($ex))
end

macro model(ex)
    return make_model(make_function(ex))
end

end

Now we import the module and use the macro:

In [ ]:
using Models

m6 = @model 2x
m6(10)

So far so good. But now let's try to include a global variable in the expression:

In [ ]:
a = 2
m7 = @model 2*a*x

In [ ]:
m7(10)

We see that it cannot find `a`. Let's see what the macro is doing:

In [ ]:
@macroexpand @model 2*a*x

We see that Julia is looking for `Models.a`, i.e. a variable `a` defined inside the `Models` module. 

To fix this problem, we must write a "hygienic" macro, by **escaping** the code. This is a mechanism telling the compiler to look for variable definitions in the scope from which the macro is called (here, the current module `Main`), rather than the scope where the macro is defined (here, the `Models` module):

In [ ]:
module Models2

export Model, @model

struct Model{F}
    f::F
end

(m::Model)(x) = m.f(x)

function make_function(ex::Expr)
    return :(x -> $ex)
end

function make_model(ex::Expr)
    return :(Model($ex))
end

macro model(ex)
    return esc(make_model(make_function(ex)))
end

end

In [ ]:
using Models2

a = 2
m8 = @model 2*a*x

In [ ]:
m8(10)

This is the final, working version of the macro.

## Conclusion

We have successfully completed our task: we have seen how to create a macro that enables a simple syntax for creating a Julia object that we can use later. 

This is the recommended pattern for creating DSLs in Julia.

For a more in-depth discussion of metaprogramming techniques and macros, see my video tutorial *Invitation to intermediate Julia*, given at JuliaCon 2016:

- Video: https://www.youtube.com/watch?v=rAxzR7lMGDM
- Jupyter notebooks: https://github.com/dpsanders/intermediate_julia

**Author**: [David P. Sanders](http://sistemas.fciencias.unam.mx/~dsanders/), Associate Professor, Department of Physics, Faculty of Sciences, National University of Mexico (UNAM).